In [ ]:

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime as dt
from datetime import timedelta

# 팝업창 에러 제어
from selenium.common.exceptions import NoAlertPresentException
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import time
import os
import shutil

# ---다운로드 폴더에 새로운 파일이 생길때까지 기다리는 함수에 적용하기 위해서 자신의 다운로드 폴더 경로를 하기!
# # 예시: DOWNLOAD_DIR = "C:/Users/YourName/Downloads"
DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"



driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

def wait_for_download_complete(download_dir, timeout=30):
    """다운로드 폴더에 새로운 파일이 생길 때까지 대기"""
    before_files = set(os.listdir(download_dir))
    elapsed = 0
    while elapsed < timeout:
        time.sleep(1)
        after_files = set(os.listdir(download_dir))
        new_files = after_files - before_files
        if new_files:
            print(f"✅ 다운로드 완료: {new_files}")
            return list(new_files)[0]  # 새로 생긴 파일 이름 반환
        elapsed += 1
    raise TimeoutError("다운로드가 완료되지 않았습니다.")




def click_detail_button(driver, wait):
    """조회 결과에서 사고현황(세부) 버튼 클릭 → 팝업 열림"""
    detailed_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_box.type02 > a.btn.morebtn"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", detailed_btn)
    driver.execute_script("arguments[0].click();", detailed_btn)
    


def handle_popup_download(driver, wait, year_val, sido_text, jijace_text):
    """팝업 창 전환 → 다운로드 → 닫기 → 메인창 복귀"""
    main_window = driver.current_window_handle

    # 팝업으로 전환
    wait.until(lambda d: len(d.window_handles) > 1)
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            break

    time.sleep(3)

    # 다운로드 버튼 클릭
    download_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_box.type02.mar_t0 > a"))
    )
    driver.execute_script("arguments[0].click();", download_btn)
    
    # 다운로드 완료 대기
    downloaded_file = wait_for_download_complete(DOWNLOAD_DIR, timeout=60)


    # 팝업 닫기 후 메인으로 복귀
    driver.close()
    driver.switch_to.window(main_window)

    




In [ ]:


# 드롭다운 요소 찾기
year_select = Select(driver.find_element(By.ID, "Year"))
years = year_select.options

# 2014~2023 범위만 value로 가져오기
filtered_years = sorted([
    opt.get_attribute("value") 
    for opt in years 
    if 2016<= int(opt.get_attribute("value")) <= 2023
], key=lambda x: int(x))  # 숫자 순 정렬

# 순서대로 선택
for year_value in filtered_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    # --- 광역 반복 ---
    sido_select = Select(driver.find_element(By.ID, "sido"))
    sidos = [opt.get_attribute("value") for opt in sido_select.options if opt.get_attribute("value")]

    for sido_val in sidos:
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_value(sido_val)
        sido_text = sido_select.first_selected_option.text

        # --- 기초 반복 ---
        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        jijace_select = Select(driver.find_element(By.ID, "jijace"))
        jijaces = [opt.get_attribute("value") for opt in jijace_select.options if opt.get_attribute("value")]

        for jijace_val in jijaces:
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_value(jijace_val)
            jijace_text = jijace_select.first_selected_option.text

            # 조회 버튼 클릭
            download_btn = driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a")
            download_btn.click()

            ### 에러 팝업 창 처리 ###
            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            #여기에 이제 세부사항 다운받고 복귀하는 함수를 넣을 것임.
            # 2️⃣ 사고현황 버튼 클릭 함수
            click_detail_button(driver, wait)

            # 3️⃣ 팝업에서 다운로드 함수
            handle_popup_download(driver, wait, year_value, sido_text, jijace_text)

            time.sleep(2)  # 안정화 대기

            # 다운로드 대기
            #time.sleep(3)

            # 파일 이동 및 이름 변경
            #latest_file = max([os.path.join(DOWNLOAD_DIR, f) for f in os.listdir(DOWNLOAD_DIR)], key=os.path.getctime)
            #new_name = f"{year_val}_{sido_text}_{jijace_text}.xlsx"
            #shutil.move(latest_file, os.path.join(SAVE_DIR, new_name))

driver.quit()


✅ 메인 페이지 사고현황(세부) 버튼 클릭 완료
✅ 다운로드 완료: {'사고다발지점 상세정보 (5).xls'}
✅ 다운로드 완료: 2014, 서울, 강남구
✅ 메인 페이지 사고현황(세부) 버튼 클릭 완료
✅ 다운로드 완료: {'사고다발지점 상세정보 (6).xls'}
✅ 다운로드 완료: 2014, 서울, 강동구
✅ 메인 페이지 사고현황(세부) 버튼 클릭 완료
✅ 다운로드 완료: {'사고다발지점 상세정보 (7).xls'}
✅ 다운로드 완료: 2014, 서울, 강북구
✅ 메인 페이지 사고현황(세부) 버튼 클릭 완료
✅ 다운로드 완료: {'사고다발지점 상세정보 (8).xls'}
✅ 다운로드 완료: 2014, 서울, 강서구


Exception ignored in: <function Service.__del__ at 0x000001D8B7B676A0>
Traceback (most recent call last):
  File "c:\Users\mstel\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 196, in __del__
    self.stop()
  File "c:\Users\mstel\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 152, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\mstel\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mstel\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mstel\anaconda3\Lib\site-packages\selenium\webdriver\common\utils.py", line 99, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.169); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x3b1af3+62339]
	GetHandleVerifier [0x0x3b1b34+62404]
	(No symbol) [0x0x1f2123]
	(No symbol) [0x0x1e1b30]
	(No symbol) [0x0x1ff8f2]
	(No symbol) [0x0x265cbc]
	(No symbol) [0x0x280089]
	(No symbol) [0x0x25f1b6]
	(No symbol) [0x0x22e7a2]
	(No symbol) [0x0x22f644]
	GetHandleVerifier [0x0x626683+2637587]
	GetHandleVerifier [0x0x621a8a+2618138]
	GetHandleVerifier [0x0x3d856a+220666]
	GetHandleVerifier [0x0x3c8998+156200]
	GetHandleVerifier [0x0x3cf12d+182717]
	GetHandleVerifier [0x0x3b9a38+94920]
	GetHandleVerifier [0x0x3b9bc2+95314]
	GetHandleVerifier [0x0x3a4d0a+9626]
	BaseThreadInitThunk [0x0x76ad8654+36]
	RtlGetAppC